# Plantillas de prompt programáticamente

## Instalación de paquetes
Si estás corriendo este notebook en Google Colab, corre la siguiente celda para instalar los paquetes necesarios.

In [ ]:
# %pip install langchain langchain_community langchain_openai

In [1]:
# Corre esta celda solo si tienes un archivo .env configurado
from dotenv import load_dotenv
load_dotenv()

True

### Importemos las librerías necesarias

In [2]:
import os
from openai import OpenAI

  Vamos a crear un prompt con la API de ChatCompletion (/chat/completions).

In [3]:
client = OpenAI(api_key=os.getenv("LIA_API_KEY"), base_url=os.getenv("LIA_BASE_URL"))

In [7]:
prompt_input = """Escribe un mensaje que te gustaría enviar a un amigo sobre un tema que te interese."""
response = client.chat.completions.create(
  model=os.getenv("MODEL"),
  messages=[
    {"role": "system", "content": "Eres un asistente virtual respetuoso y amigable"},
    {"role": "user", "content": prompt_input}
  ],
  temperature= 0.7,
  max_tokens= 100  
)
 
print(response.choices[0].message.content)

¡Hola [Nombre del Amigo]!

Espero que estés teniendo un excelente día. Quería compartir contigo algo que me ha estado interesando mucho últimamente: la inteligencia artificial y su impacto en nuestra vida diaria. Es impresionante cómo está transformando industrias enteras, desde la salud hasta el entretenimiento. Por ejemplo, las IA ahora pueden ayudar a diagnosticar enfermedades con una precisión increíble o incluso componer música original.

Me encantaría saber qué opinas sobre todo esto. ¿Crees que la


### Descripción de los parámetros

- **model**: Se refiere al modelo de LLM que se va a utilizar, un servidor puede ofrecer varios modelos, cada uno con diferentes características y capacidades.
  
- **messages**: Un prompt se compone de una lista de mensajes, cada uno interpretado por un "rol" y con algún "contenido" con las instrucciones a seguir.

- **temperature**: Este es un coeficiente de "creatividad" o "entropía", que varía de 0 (sin creatividad, lo que significa que el resultado será siempre el mismo dado el mismo prompt) a 1 (el resultado variará considerablemente en cada ejecución del mismo prompt).

- **max_tokens**: Esto limita el número de tokens de salida, generalmente para reducir costos.

## Hagamos el prompt con el objeto de Langchain

In [8]:
from langchain_openai import ChatOpenAI 

llm = ChatOpenAI(
    model=os.getenv("MODEL"),
    openai_api_key=os.getenv("LIA_API_KEY"),
    openai_api_base=os.getenv("LIA_API_BASE"),
    max_tokens=2000,
    temperature=0.6,
)

In [9]:
prompt_input = """Escribe un mensaje que te gustaría enviar a un amigo sobre un tema que te interese."""
 
response = llm.invoke(prompt_input)
print(response.content)

¡Hola [Nombre del Amigo]!

Espero que estés muy bien. Quería contarte sobre un tema que me tiene súper entusiasmado últimamente: la exploración espacial y los avances recientes en esta área. ¿Sabías que hay planes para volver a la Luna en los próximos años con el programa Artemis de la NASA? ¡Es increíble pensar en lo que podríamos descubrir!

Además, las misiones a Marte están avanzando de manera impresionante. La idea de que en un futuro no muy lejano podamos tener colonias humanas en otro planeta es fascinante. Me parece un tema apasionante porque no solo implica tecnología avanzada, sino también la posibilidad de expandir nuestro conocimiento sobre el universo y nuestro lugar en él.

Me encantaría saber qué piensas al respecto y si también te interesa este tema. Quizás podríamos ver algún documental o charla sobre el tema juntos.

¡Nos hablamos pronto!

Un abrazo,
[Tu Nombre]


## Implementar una plantilla con una función

In [10]:
def generate_text_summary_prompt(text, num_words, tone):
    return f'Eres un redactor experimentado. Escribe un resumen de {num_words} palabras del siguiente texto, usando un tono {tone}: {text}'

In [12]:
turing_text = """Alan Turing (1912–1954) fue un matemático, lógico y criptógrafo británico, considerado uno de los padres de la informática moderna. Nació en Londres y desde una temprana edad mostró un talento excepcional para las matemáticas y las ciencias. Asistió a la Universidad de Cambridge, donde destacó en matemáticas puras, y más tarde obtuvo su doctorado en la Universidad de Princeton en Estados Unidos.

Turing es más conocido por su trabajo durante la Segunda Guerra Mundial en Bletchley Park, donde jugó un papel crucial en la ruptura de los códigos de la máquina Enigma, utilizada por las fuerzas alemanas para cifrar sus comunicaciones. Su invención, la "bomba de Turing", fue un dispositivo mecánico diseñado para descifrar estos códigos, y se le atribuye haber acortado la guerra en varios años y haber salvado innumerables vidas.

Además de su contribución a la criptografía, Turing hizo importantes avances en la teoría de la computación. En 1936, antes de la guerra, publicó su célebre artículo "On Computable Numbers," donde introdujo el concepto de la "máquina de Turing", un modelo teórico que describe los principios de una computadora y que es fundamental para la informática teórica. Este trabajo sentó las bases para la creación de las primeras computadoras modernas.

Turing también reflexionó sobre la inteligencia artificial, proponiendo en 1950 lo que hoy se conoce como el "Test de Turing," un experimento mental para evaluar la capacidad de una máquina para exhibir comportamiento inteligente indistinguible del de un ser humano.

A pesar de sus logros, la vida personal de Turing estuvo marcada por la tragedia. En 1952, fue procesado en Reino Unido por su homosexualidad, entonces considerada un delito. Como parte de su condena, fue sometido a una castración química, lo que afectó profundamente su salud mental. Dos años después, en 1954, Turing fue encontrado muerto en su casa, aparentemente por suicidio.

Hoy, Alan Turing es reconocido como un pionero de la informática y un héroe de guerra. Su vida y trabajo han sido revalorizados, y en 2013 recibió un indulto póstumo por su condena. Su legado sigue vivo en el campo de la tecnología, y su impacto en la ciencia y la sociedad es incalculable."""

In [13]:
input_prompt = generate_text_summary_prompt(text=turing_text, num_words=20, tone="formal y académico")
 
response = llm.invoke(input_prompt)
print(response.content)

Alan Turing, matemático británico, revolucionó la criptografía y computación. Su legado perdura, pese a la injusticia sufrida. Indultado póstumamente.


### Usando PromptTemplate

In [14]:
from langchain_core.prompts import PromptTemplate
 
prompt_template = PromptTemplate.from_template("Eres un redactor experimentado. Escribe un resumen de {num_words} palabras del siguiente texto, usando un tono {tone}: {text}")

In [15]:
prompt = prompt_template.format(text=turing_text, num_words=20, tone="formal y académico")

In [16]:
response = llm.invoke(prompt)
print(response.content)

Alan Turing, matemático británico, revolucionó la informática y la criptografía; su legado perdura pese a la injusticia sufrida.


### Ejecutando un Prompt con LCEL

In [17]:
chain = prompt_template | llm

response = chain.invoke({"text": turing_text, 
              "num_words": 20, 
              "tone": "conocedor y motivador"})

print(response.content)

Alan Turing, pionero de la informática, descifró códigos Enigma, revolucionó la computación y reflexionó sobre inteligencia artificial, dejando un legado perdurable.
